In [1]:
import json, requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import feedparser
# from markdownify import markdownify

In [49]:
def article_content(url: str) -> str:
    
    r = requests.request("GET", url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    if soup.find("div", {"class": "c-article-body main-content"}):
        
        field = soup.find("li", {"class":"c-article-subjects__item"}).text.strip()
        
        html_content = soup.find("div",{"class":"c-article-body main-content"})
        soup.find("figcaption").decompose()
        
        content = html_content.text
        img_link = "https:" + soup.find("figure",{"class":"figure"}).img["src"]
        
        return field, content, img_link
        
    return None

In [50]:
def one_week_articles() -> list:

    articles = list()
    
    nature_rss = feedparser.parse("https://www.nature.com/nature.rss")
    
    for item in nature_rss["entries"]:
        
        try:
            
            article_link = item["id"]
        
            article_date = datetime.strptime(item["updated"], '%Y-%m-%d') 

            if (datetime.today() - timedelta(weeks=1)) < article_date < datetime.today():

                _data = article_content(article_link)

                if _data is not None:

                    article = {}

                    article["title"] = item["title"]
                    article["summary"] = item["summary"]

                    field, _content, img_link = _data

                    article["field"] = field
                    article["content"] = _content
                    article["img_link"] = img_link

                    articles.append(article)
        except Exception as Error:
            
            print(f"Cant not scarpe { article_link } due to - {Error}")

    return articles

In [51]:
data = one_week_articles()

Cant not scarpe https://www.nature.com/articles/d41586-023-00839-y due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00742-6 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00762-2 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00829-0 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00785-9 due to - 'NoneType' object has no attribute 'decompose'
Cant not scarpe https://www.nature.com/articles/d41586-023-00698-7 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00746-2 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d41586-023-00814-7 due to - 'NoneType' object has no attribute 'text'


In [52]:
len(data)

19

In [13]:
nature_rss = feedparser.parse("https://www.nature.com/nature.rss")
nature_rss

{'bozo': 1,
 'entries': [],
 'feed': {'html': {'lang': 'en'},
  'meta': {'name': 'access', 'content': 'Yes'},
  'links': [{'rel': 'search',
    'href': 'https://www.nature.com/search',
    'type': 'text/html'},
   {'rel': 'search',
    'href': 'https://www.nature.com/opensearch/opensearch.xml',
    'type': 'application/opensearchdescription+xml',
    'title': 'nature.com'},
   {'rel': 'search',
    'href': 'https://www.nature.com/opensearch/request',
    'type': 'application/sru+xml',
    'title': 'nature.com'}],
  'summary': '<div class="position-relative cleared z-index-50">\n    <header>\n        <div class="hide-print container background-gray-dark">\n            <div class="content mq1200-padded">\n                <a class="skiplink contrast-text js-no-scroll" href="https://www.nature.com/nature.rss#content" id="skiplink">Skip to main content</a>\n            </div>\n        </div>\n        <div class="hide-print text-orange content grade-c-show">\n            <p>Thank you for vis

In [22]:
articles = list()

nature_rss = feedparser.parse("https://www.nature.com/nature.rss")

for item in nature_rss["entries"]:

    article_date = datetime.strptime(item["updated"], '%Y-%m-%d') 

    if (datetime.today() - timedelta(weeks=1)) < article_date < datetime.today():

        print(item["id"])

        _data = article_content(item["id"])

        if _data is not None:

            article = {}

            article["title"] = item["title"]
            article["summary"] = item["summary"]

            _data = article_content(item["id"])
                
            field, _content, img_link = _data

            article["field"] = field
            article["content"] = _content
            article["img_link"] = img_link

            articles.append(article)

https://www.nature.com/articles/d41586-023-00795-7


AttributeError: 'NoneType' object has no attribute 'decompose'

In [38]:
r = requests.get("https://www.nature.com/articles/d41586-023-00795-7")
soup = BeautifulSoup(r.content, "html.parser")

In [42]:
soup.find("figure",{"class":"figure"}).img["src"]

'//media.nature.com/w767/magazine-assets/d41586-023-00795-7/d41586-023-00795-7_24630594.jpg'

In [26]:
_test_ls = [
    "https://www.nature.com/articles/d41586-023-00831-6",
    "https://www.nature.com/articles/d41586-023-00818-3",
    "https://www.nature.com/articles/d41586-023-00816-5"
]

In [100]:
soup.find("img",{"class":"figure__image"})["src"]

'//media.nature.com/lw767/magazine-assets/d41586-023-00831-6/d41586-023-00831-6_24636118.jpg'

In [28]:
lorem = """Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."""

articles = list()
for link in _test_ls:
    _data = article_content(link)
            
    if _data is not None:

        article = {}

        article["title"] = "Sample Title"
        article["summary"] = lorem

        _data = article_content(link)

        field, _content, img_link = _data

        article["field"] = field
        article["content"] = _content
        article["img_link"] = img_link
        
        articles.append(article)

In [29]:
from jinja2 import Environment, FileSystemLoader

In [30]:
articles

[{'title': 'Sample Title',
  'summary': "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.",
  'field': 'Publishing',
  'content': '\nLast October, the pioneering life-sciences journal eLife introduced bold changes to its editorial practice — which some researchers applauded as reimagining the purpose of a scientific journal. From 31 January this year, eLife said, it would publish every paper it sent out for peer review: authors would never again receive a rejection afte

In [53]:
results_filename = "sample.html"
results_template = environment.get_template("index.html")
context = {
    "data": data,
}
with open(results_filename, mode="w", encoding="utf-8") as results:
    results.write(results_template.render(context))
    print(f"... wrote {results_filename}")

... wrote sample.html


In [35]:
import pdfkit

ModuleNotFoundError: No module named 'pdfkit'